# Computational linear algebra: Performance exploration

In this section we will learn how to perform some benchmarks on some
matrix operations to get an idea of the operations performance. This
will be done manually, but in the future you can use specialized tools
like google benchmark, <https://github.com/google/benchmark>, or
nanobench, <https://nanobench.ankerl.com/> , etc. 



You can explore some general remarks and specific techniques at
- <https://www.youtube.com/watch?v=fDAPJ7rvcUw> (mat mul)
- <https://old.reddit.com/r/cpp/comments/128tnij/which_is_the_best_way_to_work_with_matrices_and/>
- <https://youtu.be/ioJkA7Mw2-U?si=9htzDJ4vA6F3cZRO> : heap is slow
- <https://news.ycombinator.com/item?id=39714053>


In our example, we will use the `eigen c++` library. Therefore, we will start with some `eigen c++` examples.

## `Eigen c++` library

"Eigen is a C++ template library for linear algebra: matrices, vectors, numerical solvers, and related algorithms."  Taken from <http://eigen.tuxfamily.org/index.php?title=Main_Page> 

For a quick start go to: <http://eigen.tuxfamily.org/dox/GettingStarted.html>

### Creating basic matrices

The following shows how to create a dynamic 2x2 matrix of doubles (`X` means dynamic, `d` means double), and also show how to access a given element. 
```c++
#include <iostream>
#include <eigen3/Eigen/Dense>

int main(void)
{
    Eigen::MatrixXd m(2, 2);
    m(0, 1) = -9.877;
    std::cout << m << std::endl;

    return 0;
}
```
|     |        |
|-----|--------|
| 0   | -9.877 |
| 0   | 0      |

The following shows how to create a random matrix
```c++
#include <iostream>
#include <cstdlib>
#include <eigen3/Eigen/Dense>

int main(int argc, char **argv)
{
    //const int SEED = std::atoi(argv[1]);
    //srand(SEED); // control seed
    Eigen::MatrixXd m = Eigen::MatrixXd::Random(5,5);
    std::cout << m << std::endl;

    return 0;
}
```

|           |           |            |           |           |
|-----------|-----------|------------|-----------|-----------|
| 0.680375  | -0.604897 | -0.0452059 | 0.83239   | -0.967399 |
| -0.211234 | -0.329554 | 0.257742   | 0.271423  | -0.514226 |
| 0.566198  | 0.536459  | -0.270431  | 0.434594  | -0.725537 |
| 0.59688   | -0.444451 | 0.0268018  | -0.716795 | 0.608354  |
| 0.823295  | 0.10794   | 0.904459   | 0.213938  | -0.686642 |

**Exercise**: Create a 8x8 random matrix and print it to the screen. 
Control the seed, print it twice to check that the elements are the same.

Matrix-vector multiplication is very simple with overloaded `*` operator.
``` c++
#include <iostream>
#include <eigen3/Eigen/Dense>

int main()
{
    const int N = 3;
    srand(10);
    Eigen::MatrixXd m = Eigen::MatrixXd::Random(N, N);
    m = (m + Eigen::MatrixXd::Constant(N, N, 1.2)) * 50;
    std::cout << "m =" << std::endl << m << std::endl;
    Eigen::VectorXd v(N);
    v << 1, 2, 3;
    std::cout << "m * v =" << std::endl << m * v << std::endl;
}
```

### Linear Algebra with eigen c++

In this example we are going to use eigen to solve the system
$\boldsymbol{A}\vec x = \vec{x}$ using HouseHolder QR decomposition as
done in
<http://eigen.tuxfamily.org/dox/group__TutorialLinearAlgebra.html> .

``` c++
#include <iostream>
#include <eigen3/Eigen/Dense>

int main()
{
    Eigen::Matrix3f A;
    Eigen::Vector3f b;
    A << 1,2,3,  4,5,6,  7,8,10;
    b << 3, 3, 4;
    std::cout << "Here is the matrix A:\n" << A << std::endl;
    std::cout << "Here is the vector b:\n" << b << std::endl;
    Eigen::Vector3f x = A.colPivHouseholderQr().solve(b);
    std::cout << "The solution is:\n" << x << std::endl;
    std::cout << "Verification:\n" << (A*x - b).norm() << std::endl;
}
```

|               |          |     |        |     |
|---------------|----------|-----|--------|-----|
| Here          | is       | the | matrix | A:  |
| 1             | 2        | 3   |        |     |
| 4             | 5        | 6   |        |     |
| 7             | 8        | 10  |        |     |
| Here          | is       | the | vector | b:  |
| 3             |          |     |        |     |
| 3             |          |     |        |     |
| 4             |          |     |        |     |
| The           | solution | is: |        |     |
| -2            |          |     |        |     |
| 0.999997      |          |     |        |     |
| 1             |          |     |        |     |
| Verification: |          |     |        |     |
| 1.43051e-06   |          |     |        |     |

> **Exercise**: Create a random 40x40 matrix $A$, a random vector $b$ of size 40, and solve the system $Ax = b$ using `FullPivHouseholderQR`. Compare the execution time with a 100x100 problem size.

> **Exercise:** Compile the previous example with profiling, run under valgrind cachegrind, and check the output. Can you change or optimize any of that?

Now use eigen to compute the eigenvalues of a given matrix:

``` c++
#include <iostream>
#include <eigen3/Eigen/Dense>

int main()
{
    Eigen::Matrix2d A;
    A << 1, 2, 2, 3;
    std::cout << "Here is the matrix A:\n" << A << std::endl;
    Eigen::SelfAdjointEigenSolver<Eigen::Matrix2d> eigensolver(A);
    std::cout << "The eigenvalues of A are:\n" << eigensolver.eigenvalues() << std::endl;
    std::cout << "Here's a matrix whose columns are eigenvectors of A \n"
              << "corresponding to these eigenvalues:\n"
              << eigensolver.eigenvectors() << std::endl;
}

```

> **Exercise :** Compute the eigen-values of a random matrix of size 20x20.

## Matrix operations performance: LU operations to solve `Ax = b`

In the following we want to explore the performance of the following
program as a function of the system size. We will explore the role of the compiler optimization flags, and possibly parallelization using threads. By reading <https://eigen.tuxfamily.org/dox/TopicMultiThreading.html>, we see that eigen supports parallelization for some operations, and we will take solving with partial pivote LU as example.

This is a very simple example on how to use it
``` c++
#include <iostream>
#include <eigen3/Eigen/Dense>

void solvesystem(int size);

int main(int argc, char ** argv)
{
  const int M = atoi(argv[1]); // Matrix size
  solvesystem(M);
}

void solvesystem(int size)
{
  Eigen::MatrixXd A = Eigen::MatrixXd::Random(size, size);
  Eigen::MatrixXd b = Eigen::MatrixXd::Random(size, 1);
  // crono star
  //Eigen::MatrixXd x = A.fullPivLu().solve(b);
  //Eigen::MatrixXd x = A.partialPivLu().solve(b);
  Eigen::MatrixXd x = A.lu().solve(b);
  // crono end
  double relative_error = (A*x - b).norm() / b.norm(); // norm() is L2 norm
  std::cout << A << std::endl;
  std::cout << b << std::endl;
  std::cout << "The relative error is:\n" << relative_error << std::endl;
}

// usar chrono para medir el tiempo
// perf
// remove printing
// modify printing
```

|             |           |           |           |           |
|-------------|-----------|-----------|-----------|-----------|
| -0.999984   | -0.562082 | -0.232996 | 0.0594004 | -0.165028 |
| -0.736924   | -0.905911 | 0.0388327 | 0.342299  | 0.373545  |
| 0.511211    | 0.357729  | 0.661931  | -0.984604 | 0.177953  |
| -0.0826997  | 0.358593  | -0.930856 | -0.233169 | 0.860873  |
| 0.0655345   | 0.869386  | -0.893077 | -0.866316 | 0.692334  |
| 0.0538576   |           |           |           |           |
| -0.81607    |           |           |           |           |
| 0.307838    |           |           |           |           |
| -0.168001   |           |           |           |           |
| 0.402381    |           |           |           |           |
| The         | relative  | error     | is:       |           |
| 7.87638e-16 |           |           |           |           |

Now, to be able to call this in a parametric way, we can put the solve into a function

```c++
#include <iostream>
#include <eigen3/Eigen/Dense>

void solvesystem(int size);

int main(int argc, char ** argv)
{
  const int M = atoi(argv[1]); // Matrix size
  solvesystem(M);
}

void solvesystem(int size)
{
  Eigen::MatrixXd A = Eigen::MatrixXd::Random(size, size);
  Eigen::MatrixXd b = Eigen::MatrixXd::Random(size, 1);
  // crono star
  Eigen::MatrixXd x = A.fullPivLu().solve(b);
  // crono end
  // double relative_error = (A*x - b).norm() / b.norm(); // norm() is L2 norm
  //std::cout << A << std::endl;
  //std::cout << b << std::endl;
  //std::cout << "The relative error is:\n" << relative_error << std::endl;
}

// usar chrono para medir el tiempo
// gprof / perf
// remove printing
// modify testing
```

If you use just

``` bash
time ./a.out
```

you will be measuring the whole execution time, including the memory
allocation, the actual operation, and the memory freeing. It is much
better to actually measure only what we are interesting in, which is the
actual memory operation.

To create a reasonable benchmark, we need to

-   Fill the matrix randomly, controlling the seed
-   Measure only processing time (wall and cpu times)
-   Statistics
-   Print the time as function of N and plot (run this in the shell)
-   Use threads (openmp)

---
**Filling the matrix randomly**

For repeatability, we can control the seed with the `srand` instructions
(check the manual, and also
<https://stackoverflow.com/questions/67387539/set-random-seed-in-eigen?noredirect=1&lq=1>
and
<https://stackoverflow.com/questions/21292881/matrixxfrandom-always-returning-same-matrices>
).

---
**Measuring processing time only (both wall and cpu time)**

To measure only the times associated with the processing , we will use
the `chrono` library (<https://en.cppreference.com/w/cpp/chrono>).
Before proceeding, please read first the documentation of
`system_clock`, `steady_clock`, and `high_resolution_clock`, and decide
which one to use. Also, consider if they measure the cpu time. If not,
how to measure it? Check:

-   <https://stackoverflow.com/questions/20167685/measuring-cpu-time-in-c>
-   <https://en.cppreference.com/w/cpp/chrono/c/clock>

---
**Statistics**

We will need a function to repeat the measured function several times, and then compute both the mean, 
\begin{equation}
\bar x = \dfrac{1}{N} \sum x_i,
\end{equation}
 and the standard deviation,
\begin{equation}
\sigma = \sqrt{\dfrac{1}{N-1}\sum (x_i - \bar x)^2}.
\end{equation}

Your main function must be something  like
```c++
    #include <iostream>
    #include <chrono>
    #include <ctime>
    #include <cmath>
    #include <eigen3/Eigen/Dense>

   void stats(const int size, const int reps, // input parameters
               double & mean_wtime, double & sigma_wtime, // output
               double & mean_ctime, double & sigma_ctime); // output

    int main(int argc, char ** argv)
    {
      const int M = atoi(argv[1]); // Matrix size
      const int R = atoi(argv[2]); // Repetitions

      double mean_wtime, sigma_wtime;
      double mean_ctime, sigma_ctime;
      stats(M, R, mean_wtime, sigma_wtime, mean_ctime, sigma_ctime);
      std::cout << M
                << "\t" << mean_wtime << "\t" << sigma_wtime
                << "\t" << mean_ctime << "\t" << sigma_ctime
                << std::endl;
        
      return 0;
    }
```

> **Exercise**: Create a function that receives the size and the number of repetitions, and performs the corresponding call to the kernel function and computes and prints the stats. 


```c++
   void stats(const int size, const int reps,
               double & mean_wtime, double & sigma_wtime,
               double & mean_ctime, double & sigma_ctime)
    {
      double wsum = 0, wsum2 = 0, csum = 0, csum2 = 0;
      double wtime = 0, ctime = 0;
// TODO
      std::cout << M
          << "\t" << mean_wtime << "\t" << sigma_wtime
          << "\t" << mean_ctime << "\t" << sigma_ctime
          << std::endl;
    }

```

---
### Threads

Eigen c++ includes parallelization with openmp threads. To do so, you
need to compile with the `-fopenmp` flag and then execute as

``` bash
OMP_NUM_THREADS=n ./a.out ...
```

where `n` is the number of threads. Check the following to see the kind
of ops that can be parallelized:
<https://eigen.tuxfamily.org/dox/TopicMultiThreading.html>

Run the code for a large enough size and check if the flag has any effect.

### Measuring time as a function of size
Now you are ready to measure both the wall and cpu time as a function of size, for at least 5 samples per size, and also for several number of threads. Start using `-O3 -fopenmp` and plot the curves for 1, 2, 4, 8 and the max threads in your pc. Then try the same with `-O0 -fopenmp`, but maybe you will not be able to use large sizes. How will you run this in parallel? take into account the memory consumption and also that `parallel -j jobs` is the number of processes which is different to the number of threads (a process can sapwn a lot of threads, and you should not launch more threads than available in your machine) 

## Exercise: Measuring time for matrix-matrix multiplication, with optimization and parallelization

Now do the same study but with matrix multiplication. Implement the
matrix matmul with eigen (see the code snippet at the end). Also implement the typical three-loop matrix multiplication (but does not use for large matrix sizes). 
Always
compile with the flags `-fopenmp -O3`. And now execute in the following
way

``` shell
OMP_NUM_THREADS=VAR ./a.out ...
```

where VAR takes values from 1, 2, 3, 4, … up the output from the command
`nproc`. This means that you are starting to use parallelization. You
must run this in a multicore system (or give your virtual machine enough
resources) with a least 8 threads. Plot the time (with error bars) as a
function of the system size, and you will have four curves on that
graph, each one for each VAR value.

Basic snippet for matmul in eigen:

``` c++
double multiply(int size)
{
  // create matrices
  Eigen::MatrixXd A = Eigen::MatrixXd::Random(size, size);
  Eigen::MatrixXd B = Eigen::MatrixXd::Random(size, size);

  auto start = std::chrono::steady_clock::now();
  auto C{A*B}; // MULTIPLY
  double tmp = C(0, 0); // use the matrix to make eigen compute it
  auto end = std::chrono::steady_clock::now();

  std::clog << tmp << std::endl; // use the matrix to make eigen compute it

  std::chrono::duration<double> elapsed_seconds = end-start;
  return elapsed_secons.count();
}
```
